In [3]:
USE ALM_TEST;
GO

CREATE OR ALTER VIEW [WORK].[vw_ProlongationAnalysis_OpenAndClosed_BalanceRub]
AS
/*
  Это представление агрегирует данные по депозитам с 2024 года.
  
  Часть A. Открытые депозиты (по DT_OPEN):
    - Отбираются депозиты, открытые в месяце (по DT_OPEN) и прожившие ≥ 10 дней.
    - Для каждого депозита определяется бакет срочности по DaysLived
      (используя таблицу [man_TermGroup], поле TERM_GROUP).
    - Определяется число пролонгаций (0/1/2/3+) через до 3 LEFT JOIN к [conrel_prolongations]
      (при этом предыдущий депозит учитывается только если он прожил ≥ 10 дней).
    - Агрегируются показатели с детализацией по: MonthEnd, сегменту (с преобразованием: 
      'Розничный бизнес' → 'Розница', 'ДЧБО' → 'ЧБО', иначе 'Без сегментации'), валюте (CUR)
      и бакету срочности.
  
  Часть B. Закрытые депозиты (выходы – по DT_CLOSE_FACT):
    - Отбираются депозиты, у которых фактическое закрытие произошло в месяце,
      с теми же фильтрами (клиент, продукт, прожили ≥ 10 дней).
    - Также для них определяется бакет срочности по DaysLived.
    - Агрегируются показатели по тем же четырём измерениям.
  
  Итог. Обе группы объединяются через FULL OUTER JOIN по ключу 
       (MonthEnd, SegmentGrouping, CurrencyGrouping, TermBucketGrouping).
       
  Для обеих частей агрегирование производится с помощью GROUP BY CUBE – что даёт итоговые строки вида
  "Все месяцы", "Все сегменты", "Все валюты", "Все бакеты" и любые комбинации, например, 
  "Все сегменты, все валюты, но конкретный бакет".
  
  Все вычисляемые агрегаты оборачиваются в ISNULL для исключения NULL.
*/
WITH
-----------------------------
-- 1) Генерация месяцев (MonthEnd) от 2024-01-31 до 2025-02-28
-----------------------------
cteMonths AS (
    SELECT CONVERT(date, '2024-01-31') AS MonthEnd
    UNION ALL
    SELECT EOMONTH(DATEADD(MONTH, 1, MonthEnd))
    FROM cteMonths
    WHERE EOMONTH(DATEADD(MONTH, 1, MonthEnd)) <= '2025-02-28'
),
-----------------------------
-- 2) Открытые депозиты (по DT_OPEN)
-----------------------------
DealsInMonth AS (
    SELECT
         M.MonthEnd,
         CAST(dc.CON_ID AS BIGINT) AS CON_ID,
         dc.SEG_NAME,
         dc.CUR,
         dc.DT_OPEN,
         dc.BALANCE_RUB,
         DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE) AS DaysLived
    FROM cteMonths M
    JOIN [LIQUIDITY].[liq].[DepositContract_all] dc
         ON dc.DT_OPEN >= DATEADD(DAY, 1, EOMONTH(M.MonthEnd, -1))
        AND dc.DT_OPEN < DATEADD(DAY, 1, M.MonthEnd)
        AND dc.CLI_SUBTYPE = 'INDIV'
        AND dc.PROD_NAME   != 'Эскроу'
        AND dc.DT_CLOSE_PLAN != '4444-01-01'
        AND DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE) >= 10
),
-----------------------------
-- 3) Присоединяем бакеты срочности для открытых депозитов
-----------------------------
DealsInMonthBucket AS (
    SELECT
         dm.MonthEnd,
         dm.CON_ID,
         dm.SEG_NAME,
         dm.CUR,
         dm.DT_OPEN,
         dm.BALANCE_RUB,
         dm.DaysLived,
         tg.TERM_GROUP AS TermBucket
    FROM DealsInMonth dm
    LEFT JOIN [ALM_TEST].[WORK].[man_TermGroup] tg
         ON dm.DaysLived >= tg.TERM_FROM
        AND dm.DaysLived <= tg.TERM_TO
),
-----------------------------
-- 4) Определяем число пролонгаций для открытых депозитов
-----------------------------
DealsWithProlong AS (
    SELECT
         dmb.MonthEnd,
         dmb.CON_ID,
         dmb.SEG_NAME,
         dmb.CUR,
         dmb.DT_OPEN,
         dmb.BALANCE_RUB,
         dmb.TermBucket,
         CASE
           WHEN p1.CON_ID IS NULL OR old1.CON_ID IS NULL THEN 0
           WHEN p2.CON_ID IS NULL OR old2.CON_ID IS NULL THEN 1
           WHEN p3.CON_ID IS NULL OR old3.CON_ID IS NULL THEN 2
           ELSE 3
         END AS ProlongCount
    FROM DealsInMonthBucket dmb
    LEFT JOIN [ALM].[ehd].[conrel_prolongations] p1
         ON p1.CON_ID = dmb.CON_ID
        AND p1.CON_REL_TYPE = 'PREVIOUS'
    LEFT JOIN [LIQUIDITY].[liq].[DepositContract_all] old1
         ON old1.CON_ID = p1.CON_ID_REL
        AND DATEDIFF(DAY, old1.DT_OPEN, old1.DT_CLOSE) >= 10
    LEFT JOIN [ALM].[ehd].[conrel_prolongations] p2
         ON p2.CON_ID = old1.CON_ID
        AND p2.CON_REL_TYPE = 'PREVIOUS'
    LEFT JOIN [LIQUIDITY].[liq].[DepositContract_all] old2
         ON old2.CON_ID = p2.CON_ID_REL
        AND DATEDIFF(DAY, old2.DT_OPEN, old2.DT_CLOSE) >= 10
    LEFT JOIN [ALM].[ehd].[conrel_prolongations] p3
         ON p3.CON_ID = old2.CON_ID
        AND p3.CON_REL_TYPE = 'PREVIOUS'
    LEFT JOIN [LIQUIDITY].[liq].[DepositContract_all] old3
         ON old3.CON_ID = p3.CON_ID_REL
        AND DATEDIFF(DAY, old3.DT_OPEN, old3.DT_CLOSE) >= 10
),
-----------------------------
-- 5) Агрегируем показатели по открытым депозитам с использованием GROUP BY CUBE
-----------------------------
OpenAggregated AS (
    SELECT 
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END AS SegmentGrouping,
         CUR AS CurrencyGrouping,
         TermBucket AS TermBucketGrouping,
         COUNT(*) AS OpenedDeals,
         SUM(BALANCE_RUB) AS Summ_BalanceRub,
         SUM(CASE WHEN ProlongCount > 0 THEN 1 ELSE 0 END) AS Count_Prolong,
         SUM(CASE WHEN ProlongCount = 1 THEN 1 ELSE 0 END) AS Count_1yProlong,
         SUM(CASE WHEN ProlongCount = 2 THEN 1 ELSE 0 END) AS Count_2yProlong,
         SUM(CASE WHEN ProlongCount >= 3 THEN 1 ELSE 0 END) AS Count_3plusProlong,
         SUM(CASE WHEN ProlongCount > 0 THEN BALANCE_RUB ELSE 0 END) AS Sum_ProlongRub,
         SUM(CASE WHEN ProlongCount = 1 THEN BALANCE_RUB ELSE 0 END) AS Sum_1yProlong_Rub,
         SUM(CASE WHEN ProlongCount = 2 THEN BALANCE_RUB ELSE 0 END) AS Sum_2yProlong_Rub,
         SUM(CASE WHEN ProlongCount >= 3 THEN BALANCE_RUB ELSE 0 END) AS Sum_3plusProlong_Rub
    FROM DealsWithProlong
    GROUP BY CUBE (
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END,
         CUR,
         TermBucket
    )
),
-----------------------------
-- 6) Закрытые депозиты (выходы) – отбираются по DT_CLOSE_FACT
-----------------------------
ClosedDealsInMonth AS (
    SELECT
         M.MonthEnd,
         CAST(dc.CON_ID AS BIGINT) AS CON_ID,
         dc.SEG_NAME,
         dc.CUR,
         dc.DT_OPEN,
         dc.DT_CLOSE_FACT,
         dc.BALANCE_RUB,
         DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE) AS DaysLived
    FROM cteMonths M
    JOIN [LIQUIDITY].[liq].[DepositContract_all] dc
         ON dc.DT_CLOSE_FACT >= DATEADD(DAY, 1, EOMONTH(M.MonthEnd, -1))
         AND dc.DT_CLOSE_FACT < DATEADD(DAY, 1, M.MonthEnd)
         AND dc.CLI_SUBTYPE = 'INDIV'
         AND dc.PROD_NAME != 'ЭскроУ'
         AND dc.DT_CLOSE_PLAN != '4444-01-01'
         AND DATEDIFF(DAY, dc.DT_OPEN, dc.DT_CLOSE) >= 10
),
-----------------------------
-- 7) Присоединяем бакеты срочности для закрытых депозитов
-----------------------------
ClosedDealsInMonthBucket AS (
    SELECT
         cdm.MonthEnd,
         cdm.CON_ID,
         cdm.SEG_NAME,
         cdm.CUR,
         cdm.DT_OPEN,
         cdm.DT_CLOSE_FACT,
         cdm.BALANCE_RUB,
         cdm.DaysLived,
         tg.TERM_GROUP AS TermBucket
    FROM ClosedDealsInMonth cdm
    LEFT JOIN [ALM_TEST].[WORK].[man_TermGroup] tg
         ON cdm.DaysLived >= tg.TERM_FROM
         AND cdm.DaysLived <= tg.TERM_TO
),
-----------------------------
-- 8) Агрегируем показатели по закрытым депозитам с использованием GROUP BY CUBE
-----------------------------
ClosedAggregated AS (
    SELECT
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END AS SegmentGrouping,
         CUR AS CurrencyGrouping,
         TermBucket AS TermBucketGrouping,
         COUNT(*) AS ClosedDeals,
         SUM(BALANCE_RUB) AS Summ_ClosedBalanceRub
    FROM ClosedDealsInMonthBucket
    GROUP BY CUBE (
         MonthEnd,
         CASE 
           WHEN SEG_NAME = 'Розничный бизнес' THEN N'Розница'
           WHEN SEG_NAME = 'ДЧБО' THEN N'ЧБО'
           ELSE N'Без сегментации'
         END,
         CUR,
         TermBucket
    )
)
-----------------------------
-- 9) Объединяем агрегаты по открытым и закрытым депозитам
-----------------------------
SELECT
    COALESCE(o.MonthEnd, c.MonthEnd) AS MonthEnd,
    ISNULL(COALESCE(o.SegmentGrouping, c.SegmentGrouping), N'Все сегменты') AS SegmentGrouping,
    ISNULL(COALESCE(o.CurrencyGrouping, c.CurrencyGrouping), N'Все валюты') AS CurrencyGrouping,
    ISNULL(COALESCE(o.TermBucketGrouping, c.TermBucketGrouping), N'Все бакеты') AS TermBucketGrouping,
    ISNULL(o.OpenedDeals, 0) AS OpenedDeals,
    ISNULL(o.Summ_BalanceRub, 0) AS Summ_BalanceRub,
    ISNULL(o.Count_Prolong, 0) AS Count_Prolong,
    ISNULL(o.Count_1yProlong, 0) AS Count_1yProlong,
    ISNULL(o.Count_2yProlong, 0) AS Count_2yProlong,
    ISNULL(o.Count_3plusProlong, 0) AS Count_3plusProlong,
    ISNULL(o.Sum_ProlongRub, 0) AS Sum_ProlongRub,
    ISNULL(o.Sum_1yProlong_Rub, 0) AS Sum_1yProlong_Rub,
    ISNULL(o.Sum_2yProlong_Rub, 0) AS Sum_2yProlong_Rub,
    ISNULL(o.Sum_3plusProlong_Rub, 0) AS Sum_3plusProlong_Rub,
    CASE WHEN ISNULL(o.Summ_BalanceRub, 0) > 0 THEN
         1.0 * o.Sum_ProlongRub / o.Summ_BalanceRub
         ELSE 0 END AS [Доля_пролонгаций_Rub],
    CASE WHEN ISNULL(o.Summ_BalanceRub, 0) > 0 THEN
         1.0 * o.Sum_1yProlong_Rub / o.Summ_BalanceRub
         ELSE 0 END AS [Доля_1йПролонгации_Rub],
    CASE WHEN ISNULL(o.Summ_BalanceRub, 0) > 0 THEN
         1.0 * o.Sum_2yProlong_Rub / o.Summ_BalanceRub
         ELSE 0 END AS [Доля_2йПролонгации_Rub],
    CASE WHEN ISNULL(o.Summ_BalanceRub, 0) > 0 THEN
         1.0 * o.Sum_3plusProlong_Rub / o.Summ_BalanceRub
         ELSE 0 END AS [Доля_3plusПролонгаций_Rub],
    ISNULL(c.ClosedDeals, 0) AS ClosedDeals,
    ISNULL(c.Summ_ClosedBalanceRub, 0) AS Summ_ClosedBalanceRub,
    CASE WHEN ISNULL(o.Summ_BalanceRub, 0) > 0 THEN
         1.0 * c.Summ_ClosedBalanceRub / o.Summ_BalanceRub
         ELSE 0 END AS [Доля_выходов_Rub]
FROM OpenAggregated o
FULL OUTER JOIN ClosedAggregated c
    ON o.MonthEnd = c.MonthEnd
   AND o.SegmentGrouping = c.SegmentGrouping
   AND o.CurrencyGrouping = c.CurrencyGrouping
   AND o.TermBucketGrouping = c.TermBucketGrouping
ORDER BY
    COALESCE(o.MonthEnd, c.MonthEnd);
GO


SyntaxError: invalid character '≥' (U+2265) (112111609.py, line 10)